In [16]:
import json
import os
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","PROXY_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
os.environ["UNSKRIPT_REDIS_HOST"] = "redis-master.redis.svc.cluster.local"
os.environ["UNSKRIPT_TOKEN"] = "5c4a5754-0600-11ec-9a03-0242ac130003"
os.environ["UNSKRIPT_SIDECAR_URL"] = "http://sidecar.sidecar.svc.cluster.local"
os.environ["TENANT_URL"] = env["TENANT_URL"]
paramDict = {"sitemap": "https://www.unskript.com/sitemap.xml"}
unSkriptOutputParamDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
sitemap = nbParamsObj.get('sitemap')


w = Workflow(env, secret_store_cfg, None, global_vars=globals(), check_uuids=None)

In [17]:
pip install xmltodict textstat bs4

In [18]:
import requests
import xmltodict
import json

#This Action reads in the Sitemap, converts the XML to a dictionary, 
# and then extracts every URL into a list


response = requests.get(sitemap)
contents = response.text
# Parse the XML data to a dictionary
xml_dict = xmltodict.parse(contents)
#print(xml_dict['urlset'])
urlList = []
for url in xml_dict['urlset']['url']:
    urlList.append(url['loc'])

print("sitemap read in, list of urls created")


In [19]:
import requests

urls = urlList
#urls = ["https://unskript.com"]
sitemapResponses=[]
for url in urls:
    # get the text of the file
    response = requests.get(url)
    data = { url: response.status_code}
    sitemapResponses.append(data)
    if response.status_code != 200:
        print(url, response.status_code)

In [23]:
import requests
import textstat
from bs4 import BeautifulSoup


urls = urlList
#urls = ["https://unskript.com"]
links = {}

for url in urls:
    # get the text of the file
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    for link in soup.find_all('a'):
        link_url = link.get('href')
        if (link_url not in links) and (link_url[0:4] == "http") and ("localhost" not in link_url)and ("127.0.0.1" not in link_url) and ("runbooks.sh" not in link_url):
            #print(link_url, url)
            #we want to add it
            link_response = requests.get(link_url)
            link_status = link_response.status_code
            data = {"status": link_status, "first_seen": url}
            links[link_url] = data
print("list completed")

In [24]:
#take the list into a dataframe
import pandas as pd
import matplotlib.pyplot as plt


# Create a DataFrame from the list of dictionaries
df = pd.DataFrame.from_dict(links, orient='index')
df.tail()
#Get all urls here status is not 200
mask = df['status'] != 200
filtered_df = df[mask]

# Extract the URLs from the index of the filtered DataFrame
urls = filtered_df.index.tolist()

for index, row in filtered_df.iterrows():
    print(index, row['status'], row['first_seen'])

